In [ ]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius

    def __call__(self, value):
        return math.pi * self.radius**2 * (value or 1)  # 如果value为None，则默认计算面积，否则乘以value
    @property
    def points(self):
        return self(None)

# 创建一个半径为5的圆
my_circle = Circle(5)

# 调用__call__方法，传入None作为参数，相当于计算面积
print(my_circle.points)  # 输出: 78.53981633974483

# 调用__call__方法，传入3作为参数，相当于计算圆面积并乘以3
print(my_circle(3))  # 输出: 235.61944901923448




In [ ]:
class MyModel(nn.Module):
    def forward(self, x):
        return x

model = MyModel()
output = model.forward(input_data, additional_param)


In [1]:
# SPDX-FileCopyrightText: Copyright (c) 2021-2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
import torch
# import pytest
import unittest
import numpy as np
from torch import nn
import torch.nn.functional as F
import sys
sys.path.insert(0, 'D:\sionna-main')
try:
    import comcloak
except ImportError as e:
    import sys
    sys.path.append("../")
# from sionna.mimo import StreamManagement
# from .ofdm_test_module import *

# GPU configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Number of GPUs available :', torch.cuda.device_count())
if torch.cuda.is_available():
    gpu_num = 0  # Number of the GPU to be used
    print('Only GPU number', gpu_num, 'used.')

from comcloak.ofdm import OFDMModulator, OFDMDemodulator,\
    count_errors, count_block_errors, pam, pam_gray, qam, QAMSource, SymbolSource, BinarySource, Mapper, Constellation

class TestOFDMModulator(unittest.TestCase):
    def test_cyclic_prefixes(self):
        batch_size = 64
        fft_size = 72
        num_ofdm_symbols = 14
        qam_source = QAMSource(4)
        for cp_length in range(1, fft_size + 1):
            modulator = OFDMModulator(cp_length)
            x = qam_source([batch_size, num_ofdm_symbols, fft_size])
            x_time = modulator(x)
            x_time = torch.reshape(x_time, [batch_size, num_ofdm_symbols, -1])
            self.assertTrue(torch.equal(x_time[..., :cp_length], x_time[..., -cp_length:]))

        cp_length = fft_size + 1
        modulator = OFDMModulator(cp_length).to(device)
        x = qam_source([batch_size, num_ofdm_symbols, fft_size])
        with self.assertRaises(RuntimeError):
            x_time = modulator(x)

    def test_higher_dimensions(self):
        batch_size = [64, 12, 6]
        fft_size = 72
        num_ofdm_symbols = 14
        qam_source = QAMSource(4)
        for cp_length in range(1, fft_size + 1):
            modulator = OFDMModulator(cp_length)
            x = qam_source(batch_size + [num_ofdm_symbols, fft_size])
            x_time = modulator(x)
            x_time = torch.reshape(x_time, batch_size + [num_ofdm_symbols, -1])
            self.assertTrue(torch.equal(x_time[..., :cp_length], x_time[..., -cp_length:]))

if __name__ == '__main__':
    unittest.main()


Number of GPUs available : 1
Only GPU number 0 used.


usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\30951\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-20492mr77GLs6eTu4.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
#
# SPDX-FileCopyrightText: Copyright (c) 2021-2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
import sys
sys.path.insert(0, 'D:\sionna-main')
try:
    import sionna
except ImportError as e:
    import sys
    sys.path.append("../")

from sionna.ofdm import OFDMModulator, OFDMDemodulator, ResourceGrid, ResourceGridMapper, ResourceGridDemapper
from sionna.mimo import StreamManagement
from sionna.utils import QAMSource
from tensorflow.keras import Model

# import pytest
import unittest
import numpy as np
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Number of the GPU to be used
    try:
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)


class TestOFDMModulator(unittest.TestCase):
    def test_cyclic_prefixes(self):
        batch_size = 64
        fft_size = 72
        num_ofdm_symbols = 14
        qam_source = QAMSource(4)
        for cp_length in range(1,fft_size+1):
            modulator = OFDMModulator(cp_length)
            x = qam_source([batch_size, num_ofdm_symbols, fft_size])
            x_time = modulator(x)
            x_time = tf.reshape(x_time, [batch_size, num_ofdm_symbols, -1])
            self.assertTrue(np.array_equal(x_time[...,:cp_length], x_time[...,-cp_length:]))

        cp_length = fft_size+1
        modulator = OFDMModulator(cp_length)
        x = qam_source([batch_size, num_ofdm_symbols, fft_size])
        with self.assertRaises(AssertionError):
            x_time = modulator(x)

    def test_higher_dimensions(self):
        batch_size = [64, 12, 6]
        fft_size = 72
        num_ofdm_symbols = 14
        qam_source = QAMSource(4)
        for cp_length in range(1,fft_size+1):
            modulator = OFDMModulator(cp_length)
            x = qam_source(batch_size + [num_ofdm_symbols, fft_size])
            x_time = modulator(x)
            x_time = tf.reshape(x_time, batch_size + [num_ofdm_symbols, -1])
            self.assertTrue(np.array_equal(x_time[...,:cp_length], x_time[...,-cp_length:]))

unittest.main()

In [3]:

import tensorflow as tf

class MyLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super(MyLayer, self).__init__()
        self.units = units
        print(units)

    def build(self, input_shape):
        print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        print(inputs)
        return tf.matmul(inputs, self.w) + self.b

# 使用自定义层
layer = MyLayer(10)
output = layer(tf.random.uniform((5, 5)))
print(output)

10
(5, 5)
tf.Tensor(
[[0.7665863  0.2483437  0.65044546 0.34034836 0.5344738 ]
 [0.5768297  0.8137628  0.8645096  0.37304568 0.18203628]
 [0.43702555 0.2144996  0.70758843 0.10825706 0.85274196]
 [0.72437906 0.41653347 0.71011627 0.5930704  0.52434206]
 [0.1442895  0.7660911  0.6488236  0.06977487 0.5795363 ]], shape=(5, 5), dtype=float32)
tf.Tensor(
[[ 0.07614294 -0.0377135  -0.05135499  0.0455152  -0.0099695  -0.02818357
  -0.02651541 -0.07205104 -0.05843713  0.06963677]
 [ 0.06449924  0.00659409  0.02167855  0.1076876   0.04814871  0.00756671
  -0.01982652 -0.03567904 -0.06240568  0.06445968]
 [ 0.02913768 -0.04475894 -0.10159616  0.0083041  -0.01466483 -0.05252483
  -0.04925257 -0.1498481  -0.08364629  0.09769362]
 [ 0.08463693 -0.02210855 -0.04019443  0.07286407  0.01763053 -0.02113176
  -0.01932536 -0.05856701 -0.06242862  0.07186877]
 [-0.01050909  0.00242789 -0.02867445  0.07862233  0.03525058 -0.00272689
  -0.04480264 -0.09863007 -0.07220872  0.07621948]], shape=(5, 10), dtype

In [7]:
import torch

# 原始二维张量
tensor = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype=torch.float)

# 要更新的位置索引，二维
indices = torch.tensor([
    [0, 1],
    [2, 1]
])

# 更新值，与indices的形状相同
updates = torch.tensor([10, 30], dtype=torch.float)

# 使用高级索引进行替换
tensor[indices[:, 0], indices[:, 1]] = updates

# 打印更新后的张量
print(tensor)


tensor([[ 1., 10.,  3.],
        [ 4.,  5.,  6.],
        [ 7., 30.,  9.]])


In [5]:
import torch

def update_tensor(tensor, indices, updates):
    
    # 使用高级索引进行更新
    tensor[tuple(indices.t())] = updates
    print(tuple(indices.t()))
    return tensor

# 示例用法

# 原始张量 (四维)
tensor = torch.tensor([
    [
        [
            [1, 2, 3], 
            [4, 5, 6]
        ], 
        [
            [7, 8, 9], 
            [10, 11, 12]
        ]
    ],
    [
        [
            [13, 14, 15], 
            [16, 17, 18]
        ], 
        [
            [19, 20, 21], 
            [22, 23, 24]
        ]
    ]
], dtype=torch.float)

# 要更新的位置索引 (num_updates, num_dimensions)
indices = torch.tensor([
    [0, 1, 1],
    [1, 1, 0]
])

# 更新值 (num_updates,)
updates = torch.tensor([[100, 200, 300],[100, 200, 300]], dtype=torch.float)

# 更新张量
updated_tensor = update_tensor(tensor, indices, updates)

# 打印更新后的张量
print(updated_tensor)


(tensor([0, 1]), tensor([1, 1]), tensor([1, 0]))
tensor([[[[  1.,   2.,   3.],
          [  4.,   5.,   6.]],

         [[  7.,   8.,   9.],
          [100., 200., 300.]]],


        [[[ 13.,  14.,  15.],
          [ 16.,  17.,  18.]],

         [[100., 200., 300.],
          [ 22.,  23.,  24.]]]])


In [1]:
import torch

def update_tensor(tensor, indices, updates):
    # 使用高级索引进行更新
    tensor[tuple(indices.t())] = updates
    print(tuple(indices.t()))
    return tensor

# 示例用法

# 原始张量 (五维)
tensor = torch.tensor([
    [
        [
            [
                [1, 2, 3], 
                [4, 5, 6]
            ], 
            [
                [7, 8, 9], 
                [10, 11, 12]
            ]
        ],
        [
            [
                [13, 14, 15], 
                [16, 17, 18]
            ], 
            [
                [19, 20, 21], 
                [22, 23, 24]
            ]
        ]
    ],
    [
        [
            [
                [25, 26, 27], 
                [28, 29, 30]
            ], 
            [
                [31, 32, 33], 
                [34, 35, 36]
            ]
        ],
        [
            [
                [37, 38, 39], 
                [40, 41, 42]
            ], 
            [
                [43, 44, 45], 
                [46, 47, 48]
            ]
        ]
    ]
], dtype=torch.float)

# 要更新的位置索引 (num_updates, num_dimensions)
indices = torch.tensor([
    [0, 0, 1, 1],  # 更新 (0, 0, 1, 1, 0) 位置的值
    [1, 1, 0, 0]   # 更新 (1, 1, 0, 0, 2) 位置的值
])

# 更新值 (num_updates,)
updates = torch.tensor([[100, 200, 300],[100, 200, 300]], dtype=torch.float)

# 更新张量
updated_tensor = update_tensor(tensor, indices, updates)

# 打印更新后的张量
print(updated_tensor)


(tensor([0, 1]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]))
tensor([[[[[  1.,   2.,   3.],
           [  4.,   5.,   6.]],

          [[  7.,   8.,   9.],
           [100., 200., 300.]]],


         [[[ 13.,  14.,  15.],
           [ 16.,  17.,  18.]],

          [[ 19.,  20.,  21.],
           [ 22.,  23.,  24.]]]],



        [[[[ 25.,  26.,  27.],
           [ 28.,  29.,  30.]],

          [[ 31.,  32.,  33.],
           [ 34.,  35.,  36.]]],


         [[[100., 200., 300.],
           [ 40.,  41.,  42.]],

          [[ 43.,  44.,  45.],
           [ 46.,  47.,  48.]]]]])


In [1]:
import tensorflow as tf


# 原始张量 (四维)
tensor = tf.cast([
    [
        [
            [
                [1, 2, 3], 
                [4, 5, 6]
            ], 
            [
                [7, 8, 9], 
                [10, 11, 12]
            ]
        ],
        [
            [
                [13, 14, 15], 
                [16, 17, 18]
            ], 
            [
                [19, 20, 21], 
                [22, 23, 24]
            ]
        ]
    ],
    [
        [
            [
                [25, 26, 27], 
                [28, 29, 30]
            ], 
            [
                [31, 32, 33], 
                [34, 35, 36]
            ]
        ],
        [
            [
                [37, 38, 39], 
                [40, 41, 42]
            ], 
            [
                [43, 44, 45], 
                [46, 47, 48]
            ]
        ]
    ]
], dtype=tf.int32)

# 要更新的位置索引 (num_updates, num_dimensions)
indices = tf.cast([
    [0, 0, 1, 1],  # 更新 (0, 0, 1, 1, 0) 位置的值
    [1, 1, 0, 0]   # 更新 (1, 1, 0, 0, 2) 位置的值
], dtype=tf.int32)

# 更新值 (num_updates,)
updates = tf.cast([[100, 200, 300],[100, 200, 300]], dtype=tf.int32)
updated_tensor = tf.tensor_scatter_nd_update(tensor, indices, updates)
print(updated_tensor)

tf.Tensor(
[[[[[  1   2   3]
    [  4   5   6]]

   [[  7   8   9]
    [100 200 300]]]


  [[[ 13  14  15]
    [ 16  17  18]]

   [[ 19  20  21]
    [ 22  23  24]]]]



 [[[[ 25  26  27]
    [ 28  29  30]]

   [[ 31  32  33]
    [ 34  35  36]]]


  [[[100 200 300]
    [ 40  41  42]]

   [[ 43  44  45]
    [ 46  47  48]]]]], shape=(2, 2, 2, 2, 3), dtype=int32)


In [18]:
import torch
def gather_pytorch(input_data, indices=None, batch_dims=0, axis=0):
    input_data = torch.tensor(input_data)
    indices = torch.tensor(indices)
    if batch_dims == 0:
        if axis < 0:
            axis = len(input_data.shape) + axis
        data = torch.index_select(input_data, axis, indices.flatten())
        shape_input = list(input_data.shape)
        # shape_ = delete(shape_input, axis)
        # 连接列表
        shape_output = shape_input[:axis] + \
            list(indices.shape) + shape_input[axis + 1:]
        data_output = data.reshape(shape_output)
        return data_output
    else:
        data_output = []
        for data,ind in zip(input_data, indices):
            r = gather_pytorch(data, ind, batch_dims=batch_dims-1)
            data_output.append(r)
        return torch.stack(data_output)


params = torch.tensor([
    [0, 0, 1, 0, 2],
    [3, 0, 0, 0, 4],
    [0, 5, 0, 6, 0]])
indices = torch.tensor([
    [2, 4],
    [0, 4],
    [1, 3]])
gather_pytorch(params, indices, batch_dims=1, axis=1).numpy()

C:\Users\30951\AppData\Local\Temp\ipykernel_37700\2763354305.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_data)
C:\Users\30951\AppData\Local\Temp\ipykernel_37700\2763354305.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indices = torch.tensor(indices)


array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int64)

In [1]:
import tensorflow as tf
tensor = [[1, 1], [1, 1], [1, 1]]    # tf.rank(tensor) == 2
indices = [[0, 1], [2, 0]]           # num_updates == 2, index_depth == 2
updates = [5, 10]                    # num_updates == 2
print(tf.tensor_scatter_nd_update(tensor, indices, updates))





tf.Tensor(
[[ 1  5]
 [ 1  1]
 [10  1]], shape=(3, 2), dtype=int32)


In [5]:
import torch
tensor = [[1, 1], [1, 1], [1, 1]]    # tf.rank(tensor) == 2
indices = [[0, 1], [2, 0]]           # num_updates == 2, index_depth == 2
updates = [5, 10]                    # num_updates == 2
tensor = torch.tensor(tensor)
updates = torch.tensor(updates)
indices = torch.tensor(indices)
tensor[tuple(indices.t())] = updates
print(tensor)

tensor([[ 1,  5],
        [ 1,  1],
        [10,  1]])
